In [1]:
import pandas as pd
from eutils import Client
import time
import os

featdat = pd.read_csv("data/for_genes.txt",index_col=0) # INPUT
names = set([x.split("_")[0] for x in featdat.index])
entrezids = set()

ec = Client(api_key="f7078150f88498b5a1d0ba56db5dcffc2908")
for gene in names:
    esr = ec.esearch(db='gene',term=gene+'[PREF] AND Homo+sapiens[ORGN]')
    if(len(esr.ids) == 1):
        entrezids.add(str(esr.ids[0]))
    elif(len(esr.ids) > 1):
        #print("ALERT: multiple results found for",gene," taking first")
        entrezids.add(str(esr.ids[0]))
    else:
        #print("ALERT: no results found for",gene,"Taking more general search...")
        esr = ec.esearch(db='gene',term=gene+' AND Homo+sapiens[ORGN]') # comment out for stricter results
        if(len(esr.ids) > 1):
            #print("ALERT 2: multiple results found for",gene," taking first")
            entrezids.add(str(esr.ids[0]))
        elif(len(esr.ids) == 0):
            #print("ALERT 2: no results found for",gene," skipping")
            continue
        else:
            #print("Success")
            entrezids.add(str(esr.ids[0]))
    time.sleep(1) # pause to prevent exceeding API rate limit

with open("david_out/davidforeground-mutations.txt", 'w') as f: # OUTPUT
    for item in entrezids:
        f.write("%s\n" % item)
f.close()

featdat

""
CASP7
CTNNA2
KPNB1
ZC3H18
LATS2
ANKRD13C
...
ZBTB20
FXR1
MEX3A


In [3]:
maffolder = "data/database_UCEC/" # INPUT

# import class data
classtmp = pd.read_csv("data/cimp_classes.csv", skiprows=1, index_col=0, header=None)
classtmp.index = [x.replace('.','-') for x in classtmp.index]
crcclasses = pd.concat([classtmp, pd.read_csv("data/cimp_classes.csv", index_col=0, header=None)], axis=0)
crcclasses = crcclasses[~crcclasses.index.duplicated(keep='first')] # drop duplicates
crcclasses = crcclasses[(crcclasses[2] == "CIMP+")|(crcclasses[2] == "CIMP-")|(crcclasses[2] == "CIMPi")]

backgroundentrezs = set()
for maffile in os.listdir(maffolder):
    if(maffile[-4:] == ".maf"): # ensure it's a maf file
        fullpath = maffolder + maffile
        df = pd.read_csv(fullpath, sep='\t', skiprows=5, dtype=str)
        df["Tumor_Sample_Barcode"] = ["-".join(x.split("-")[:4])[:-1] for x in df["Tumor_Sample_Barcode"]]
        df = df[df["Tumor_Sample_Barcode"].isin(crcclasses.index)]
        backgroundentrezs.update(list(df["Entrez_Gene_Id"]))
with open("david_out/davidbackground-mutations.txt", 'w') as f: # OUTPUT
    for item in backgroundentrezs:
        f.write("%s\n" % item)
f.close()

len(backgroundentrezs)

20531